图像识别-OneStage系列
===

# 1.YoLoV1
## 1.1.YoLo
YOLO = You Only Look Once，只需要看一次，不需要类似RPN的候选框提取，直接进行整图回归就可以了

## 1.2.结构图
![images](images/02_02_01_001.png)

## 1.3.算法描述
- 将图像划分为固定的网格（比如7*7），如果一个目标的中心落入格子，则该格子就负责检测该目标
- 每个网格预测包含2个边框盒以及20个判别属性(这是针对VOC训练集而言的，VOC有20个类别)。每个边框盒包括4个坐标以及一个置信值Confidence。所以网格里面包含的数据维度就是2 * 5 + 20 = 30维
- 置信值为格子包含目标的概率与IOU的乘积
- 每个格子预测包含某一类目标的条件概率值
- 每个bounding box通过对应格子的类别概率和box置信度相乘得到该类别的置信分数。这个分数衡量了该类别出现在box中的概率以及该box与目标的吻合程度
- 网络输出层即为每个Grid的对应结果，由此实现端到端的训练。

![images](images/02_02_01_002.png)

## 1.4.损失函数
$$\begin{eqnarray}
Loss&=&\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}[(x_i-\hat{x}_i)^2+(y_i-\hat{y}_i)^2] \\
&+&\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}[(\sqrt{\omega_i}-\sqrt{\hat{\omega}_i})^2+(\sqrt{h_i}-\sqrt{\hat{h}_i})^2]\\
&+&\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}(C_i-\hat{C}_i)^2\\
&+&\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{noobj}(C_i-\hat{C_i})^2\\
&+&\sum_{i=0}^{S^2}\ell_{i}^{obj}\sum_{c \in classes}[p_i(c)-\hat{p}_i(c)]^2
\end{eqnarray}$$
对于代码[Pytorch_01.YOLOV1.py](01.YOLOv1/Pytorch_01.YOLOV1.py)来说有如下输入：
$$\begin{eqnarray}
\lambda_{coord}&=&5\\
\lambda_{noobj}&=&0.5\\
S&=&7\\
B&=&2\\
C&=&20
\end{eqnarray}$$
S就是每张图片划分成了$7*7$个格子，B就是每个格子里面有2套边框，C是类别，有20个类别的物体

### 1.4.1.类别预测损失-Classification Loss
$$\sum_{i=0}^{S^2}\ell_{i}^{obj}\sum_{c \in classes}[p_i(c)-\hat{p}_i(c)]^2$$
$\ell_{i}^{obj}$:如果第i个格子包含某个object的中心的话，那么这个值就是1，否则就是0<br/>
$p_i(c)$：当前格子所预测的类别是类别c的概率

### 1.4.2.位置预测损失-Localization Loss
$$\begin{eqnarray}
& &\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}[(x_i-\hat{x}_i)^2+(y_i-\hat{y}_i)^2] \\
&+&\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}[(\sqrt{\omega_i}-\sqrt{\hat{\omega}_i})^2+(\sqrt{h_i}-\sqrt{\hat{h}_i})^2]
\end{eqnarray}$$
$\ell_{ij}^{obj}$:如果第i个格子的第j个bbox表示这个object，那么就是1，否则就是0
$x,y,\omega,h$表示的就是bbox的坐标-左上角坐标$x,y$，宽$\omega$,长$h$

### 1.4.3.信心预测损失-Confidence Loss
$$\begin{eqnarray}
& &\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{obj}(C_i-\hat{C}_i)^2\\
&+&\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^B\ell_{ij}^{noobj}(C_i-\hat{C_i})^2\\
\end{eqnarray}$$
第一项表示含object的bbox的Confidence预测；第二项表示不含object的bbox的Confidence预测

# 2.SSD(Single Shot MultiBox Detector)
SSD算法属于one-stage方法，MultiBox指明了SSD是多框预测.SSD算法在准确度和速度(除了SSD512)上都比Yolo要好很多.Yolo算法缺点是难以检测小目标，而且定位不准，但是这几点重要改进使得SSD在一定程度上克服这些缺点,以下介绍如果没有单独说明，都是SSD300，也就是说输入的图片都是$300 \times 300$的

## 2.1.SSD与YoLo的不同
- 相比Yolo，SSD采用CNN卷积来直接进行检测，而不是像Yolo那样在全连接层之后做检测
- SSD提取了不同尺度的特征图来做检测，大尺度特征图(较靠前的特征图)可以用来检测小物体，而小尺度特征图(较靠后的特征图)用来检测大物体
- SSD采用了不同尺度和长宽比的先验框(Prior boxes, Default boxes，在Faster R-CNN中叫做锚，Anchors)

## 2.2.采用多尺度特征图用于检测
其实就是采用金字塔网络。所谓多尺度采用大小不同的特征图，CNN网络一般前面的特征图比较大，后面会逐渐采用stride=2的卷积或者pool来降低特征图大小。一个比较大的特征图和一个比较小的特征图，它们都用来做检测。这样做的好处是比较大的特征图来用来检测相对较小的目标，而小的特征图负责检测大目标，如图4所示，8x8的特征图可以划分更多的单元，但是其每个单元的先验框尺度比较小。

## 2.3.采用卷积进行检测
与Yolo最后采用全连接层不同，SSD直接采用卷积对不同的特征图来进行提取检测结果。对于形状为$m\times n \times p$的特征图，只需要采用$3\times 3 \times p$这样比较小的卷积核得到检测值。

## 2.4.设置先验框
在Yolo中，每个单元预测多个边界框，但是其都是相对这个单元本身（正方块），但是真实目标的形状是多变的，Yolo需要在训练过程中自适应目标的形状。而SSD借鉴了Faster R-CNN中anchor的理念，每个单元设置尺度或者长宽比不同的先验框，预测的边界框（bounding boxes）是以这些先验框为基准的，在一定程度上减少训练难度。一般情况下，每个单元会设置多个先验框，其尺度和长宽比存在差异，目标分别采用最适合它们形状的先验框来进行训练

### 2.4.1.先验框的匹配规则
确定训练图片中的ground truth与哪个先验框来进行匹配，与之匹配的先验框所丢应的边界框将负责预测它。在Yolo中，ground truth的中心落在哪个单元格，该单元格中与其IOU最大的边界框负责预测它。但是在SSD中却完全不一样，SSD的先验框与ground truth的匹配原则主要有两点
1. 对于图片中每个ground truth，找到与其IOU最大的先验框，该先验框与其匹配，这样，可以保证每个ground truth一定与某个先验框匹配。通常称与ground truth匹配的先验框为正样本。若一个先验框没有与任何ground truth进行匹配，那么该先验框只能与背景匹配，就是负样本。
2. 对于剩余的未匹配先验框，若某个ground truth的IOU大于某个阈值(一般是0.5)，那么该先验框也与这个ground truth进行匹配。

第一个原则保证每个ground truth都有一个先验框与之相对应，也就是说都会存在正样本，但是这个原则会导致正样本极少，负样本极多，这样正负样本极不平衡。所以有了第二个原则。但是第二个会导致的一个结果就是一个ground truth有可能对应多个先验框，这是可以的。如果多个ground truth与某个先验框 \text{IOU} 大于阈值，那么先验框只与IOU最大的那个先验框进行匹配。第二个原则一定在第一个原则之后进行。下图是一个匹配示意图，其中绿色的GT是ground truth，红色为先验框，FP表示负样本，TP表示正样本<br/>
![images](images/02_02_01_003.png)

### 2.4.2.负样本的处理
尽管一个ground truth可以与多个先验框匹配，但是ground truth相对先验框还是太少了，所以负样本相对正样本会很多。为了保证正负样本尽量平衡，SSD采用了hard negative mining，就是对负样本进行抽样，抽样时按照置信度误差（预测背景的置信度越小，误差越大）进行降序排列，选取误差的较大的top-k作为训练的负样本，以保证正负样本比例接近1:3。而对于正负样本数量不均衡的数据集(这里假设负样本数量圆圆大于正样本数量)，通常的做法有四种，可以参考[](../01.Basic/数据不均衡.ipynb)

## 2.5.损失函数
损失函数定义为位置误差(locatization loss， loc)与置信度误差(confidence loss, conf)的加权和
$$L(x,c,\ell,g)=\frac{1}{N}[L_{conf}(x,c)+\alpha L_{loc}(x,\ell,g)]$$
其中N是先验框的正样本数量。这里$x_{ij}^p \in \{1,0\}$为一个指示参数，当$x_{ij}^p=1$时表示第i个先验框与第j个ground truth匹配，并且ground truth的类别为p。c为类别置信度预测值，$\ell$为先验框所对应的边界框的位置预测值，而g是ground truth的位置参数。

### 2.5.1.位置误差$L_{loc}$
采用Smooth L1 Loss，定义如下：
$$L_{loc}(x,\ell,g)=\sum_{i \in Pos}^N \sum_{m \in \{cx,cy,w,h\}}x_{ij}^kS_{L1}(\ell_i^m-\hat{g}_j^m)$$
$\ell$为预测框，$g$为ground truth box。$(cx,cy)$为补偿后的默认框d的中心
$L_{conf}$为预测框$\ell$和ground truth box $g$的Smooth L1 Loss，$\alpha$的值通过交叉验证设置为1

### 2.5.2.置信度误差$L_{conf}$
采用Softmax Loss
$$
L_{conf}(x,c)=-\sum_{i \in Pos}^Nx_{ij}^plog(\hat{c}_j^p)-\sum_{i \in N_{eg}}log(\hat{c}_i^0)
$$
其中
$$\hat{c}_j^p=\frac{e^{c_i^p}}{\sum_pe^{c_i^p}}$$

## 2.6.anchor box的维度
SSD300网络的输出是一个包含3个值的tuple
1. Loc的信息，维度为?,8732,4
2. conf置信度信息，维度为?,8732,21
3. 概率信息，维度为8732,4

8372是怎么来的？我们知道SSD是多尺度特征检测，其中在$38 \times 38$的特征图上通过$3 \times 3$卷积层预测4个不同宽高比的box，$19 \times 19$特征图上预测6个，$10 \times 10$特征图预测6个，$5 \times 5$特征图预测6个，$3 \times 3$特征图预测4个，$1 \times 1$特征图预测4个，所以就有$38 \times 38 \times 4+19 \times 19 \times 6+10 \times 10 \times 6+5 \times 5 \times 6+3 \times 3 \times 4+4 = 8732$，这个8732是最终预测框的个数

## 2.7.anchor box的计算方法
SSD300的输入图片是$300 \times 300$的,无论特征图的尺寸多大，每个单元格的中心点的计算公式都是一样的，假设单元格的宽度为$\omega_d$，单元格的横纵坐标为$(i,j)$(i,j)一定是从零开始，并且等于特征图尺寸-1.那么当前单元格的中心点坐标就是$[\frac{j+0.5}{\omega_d}, \frac{i+0.5}{\omega_d}]$

### 2.7.1.38*38的特征图
单元格宽度为$\omega_d=\frac{300}{8}=37.5$。每个单元格上有4个default box，分为小正方形B1，大正方形B2，长方形B3，以及长方形B4，四个default box的中心点坐标可以通过上面的公式计算出来，那么四个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{30}{300}, \frac{30}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{60}{300}}, \sqrt{\omega_{min}^h \times \frac{60}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3)
\end{eqnarray}$$

### 2.7.2.19*19的特征图
单元格宽度为$\omega_d=\frac{300}{16}=18.75$。每个单元格上有6个default box，分为小正方形B1，大正方形B2，长方形B3、B4、B5，B6，6个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{60}{300}, \frac{60}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{111}{300}}, \sqrt{\omega_{min}^h \times \frac{111}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3) \\
B5&=&(w_5,h_5)=(\omega_{min}^w \times \sqrt{3}, \frac{\omega_{min}^h}{\sqrt{3}}) \\
B4&=&(w_6,h_6)=(h_5,w_5)
\end{eqnarray}$$

### 2.7.3.10*10的特征图
单元格宽度为$\omega_d=\frac{300}{32}=9.375$。每个单元格上有6个default box，分为小正方形B1，大正方形B2，长方形B3、B4、B5，B6，6个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{111}{300}, \frac{111}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{162}{300}}, \sqrt{\omega_{min}^h \times \frac{162}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3) \\
B5&=&(w_5,h_5)=(\omega_{min}^w \times \sqrt{3}, \frac{\omega_{min}^h}{\sqrt{3}}) \\
B4&=&(w_6,h_6)=(h_5,w_5)
\end{eqnarray}$$

### 2.7.4.5*5的特征图
单元格宽度为$\omega_d=\frac{300}{64}=4.6875$。每个单元格上有6个default box，分为小正方形B1，大正方形B2，长方形B3、B4、B5，B6，6个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{162}{300}, \frac{162}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{213}{300}}, \sqrt{\omega_{min}^h \times \frac{213}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3) \\
B5&=&(w_5,h_5)=(\omega_{min}^w \times \sqrt{3}, \frac{\omega_{min}^h}{\sqrt{3}}) \\
B4&=&(w_6,h_6)=(h_5,w_5)
\end{eqnarray}$$

### 2.7.5.3*3的特征图
单元格宽度为$\omega_d=\frac{300}{100}=3$。每个单元格上有4个default box，分为小正方形B1，大正方形B2，长方形B3、B4,4个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{213}{300}, \frac{213}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{264}{300}}, \sqrt{\omega_{min}^h \times \frac{264}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3)
\end{eqnarray}$$


### 2.7.6.1*1的特征图
单元格宽度为$\omega_d=\frac{300}{300}=1$。每个单元格上有4个default box，分为小正方形B1，大正方形B2，长方形B3、B4,4个default box的宽度和长度如下
$$\begin{eqnarray}
B1&=&(\omega_{min}^w,\omega_{min}^h)=(\frac{264}{300}, \frac{264}{300}) \\
B2&=&(\omega_{max}^w,\omega_{max}^h)=(\sqrt{\omega_{min}^w \times \frac{315}{300}}, \sqrt{\omega_{min}^h \times \frac{315}{300}}) \\
B3&=&(w_3,h_3)=(\omega_{min}^w \times \sqrt{2}, \frac{\omega_{min}^h}{\sqrt{2}}) \\
B4&=&(w_4,h_4)=(h_3,w_3)
\end{eqnarray}$$

# 3.YoLoV2
YoLoV1相对于Fast R-CNN，YOLO在目标定位方面错误率较高，YoLoV2的改进集中于在保持分类准确率的基础上增强定位精确度

## 3.1.YOLOV2的改进
### 3.1.1.全新网络Darknet-19
YOLOv2采用了全新的Darknet-19网络，将YOLOv1中最后的全连接层换成了卷积层。最后的特征图大小为13*13.输入为416*416

### 3.1.2.Anchors Box
采纳了Faster RCNN中的anchors box的思想，每个位置的各个anchor box都单独预测一套分类概率。anchors box数量为5。为何要引入anchors呢，因为在anchors上微调比直接去回归坐标要容易得多。

## 3.2.损失函数
YOLOv2的损失函数比YOLOv1的更复杂。
$$\begin{eqnarray}
Loss_t=\sum_{i=0}^W\sum_{j=0}^H\sum_{k=0}^A& &1_{MaxIOU \le Thresh}\lambda_{noobj} \times (-b_{ijk}^o)^2\\
&+&1_{t \le 12800}\lambda_{prior} \times \sum_{r \in (x,y,W,h)}(prior_k^r-b_{ijk}^r)^2 \\
&+&1_k^{truth}[\lambda_{coord} \times \sum_{r \in (x,y,W,h)}(truth^r-b_{ijk}^r)^2]\\
&+&1_k^{truth}[\lambda_{obj} \times (IOU_{truth}^k-b_{ijk}^o)^2]\\
&+&1_k^{truth}[\lambda_{class} \times \sum_{c=1}^C(truth^c-b_{ijk}^c)^2]
\end{eqnarray}$$

## 3.3.损失函数释义
$W,H$分别指的是特征图的宽和高，分别是13,13，A指的是先验框的数目5。

### 3.3.1.第一项
$$1_{MaxIOU \le Thresh}\lambda_{noobj} \times (-b_{ijk}^o)^2$$
loss是计算background的置信度误差，判断是否是用来预测背景的逻辑就是$1_{MaxIOU \le Thresh}$。需要先计算各个预测框和所有ground truth的IOU值，并且取最大值Max_IOU，如果该值小于一定的阈值(0.6)，那么这个预测框就标记为background，需要计算noobj的置信度误差

### 3.3.2.第二项
计算先验框与预测宽的坐标误差，但是只在前12800上计算，这项应该是在训练前期使预测框快速学习到先验框的形状。

### 3.3.3.第三项
计算与某个ground truth匹配的预测框各部分的坐标误差

### 3.3.4.第四项
计算与某个ground truth匹配的预测框各部分的置信度误差

### 3.3.5.第五项
计算与某个ground truth匹配的预测框各部分的分类误差

# 4.Focal Loss与RetinateNet
YOLO的精度不够高，这是因为它做的是稠密分类，核心问题是稠密proposal中前景和背景的极度不平衡。比如PASCAL VOC数据集中，每张图片上标注的目标可能也就几个，但是YOLO V2最后一层的输出是13*13*5,也就是845个候选目标，大量的负样本在loss中占据了很大比重，使得有用的loss不能回传回来。基于此，作者将经典的交叉熵损失做了变形，给那些易于被分类的简单例子小的权重，给不易区分的难例更大的权重。同时，作者提出了一个新的one-stage的检测器RetinaNet，达到了速度和精度很好地trade-off。

## 4.1.交叉熵损失
Focal Loss从交叉熵损失而来，二分类的交叉熵损失如下:
$$CE(p,y)=\begin{cases}
-log(p) & y=1\\
-log(1-p) & otherwise
\end{cases}$$
对应的，多分类的交叉熵损失是这样的：
$$CE(p,y)=-log(p_y)$$
![images](images/02_02_01_004.png)<br/>
如上图所示，蓝色线为交叉熵损失函数随着pt变化的曲线(pt意为ground truth，是标注类别所对应的概率)。可以看到，当概率大于.5，即认为是易分类的简单样本时，值仍然较大。这样，很多简单样本累加起来，就很可能盖住那些稀少的不易正确分类的类别

## 4.2.Focal Loss损失
为了改善类别样本分布不均衡的问题，已经有人提出了使用加上权重的交叉熵损失：
$$CE(p)=-\alpha_tlog(p_t)$$
即用参数$\alpha_t$来平衡，这组参数可以是超参数，也可以由类别的比例倒数决定。作者将其作为比较的baseline，提出了一个自适应调节的权重，即Focal Loss，定义如下：
$$FL(p_t)=-(1-p_t)^{\gamma}log(p_t)$$
在实际实验中，作者使用的是加权之后的Focal Loss，作者发现这样能够带来些微的性能提升。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)
    mask = torch.Tensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.
    if isinstance(index, Variable):
        ones = Variable(torch.Tensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)
    return mask.scatter_(1, index, ones)
class FocalLoss(nn.Module):
    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps
    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input)
        logit = logit.clamp(self.eps, 1. - self.eps)
        loss = -1 * y * torch.log(logit) # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss
        return loss.sum()

## 4.3.RetinaNet
利用Focal Loss，基于ResNet和Feature Pyramid Net(FPN)设计了一种新的one-stage检测框架，命名为RetinaNet

# 5.YOLO V3
按照时间循序来说，YOLO V3是最晚出来的，它的性能也最好，甚至比SSD和RetinaNet都要好。YOLOv3使用了Darknet-53网络